# HaMLeT

## Session 5: Neural Networks
by Raphael Kolk

### Goal of this Session

In this session you will, step by step, implement a neural network yourself without using any deep learning libraries. You should already be familiar with Python as well as NumPy (a package for scientific computing with Python).

### Tasks

**Task 0:** Please execute the following cell which is a workaround for data loading and ignore it until further notice.

In [ ]:
from six.moves import urllib
class MNIST:
    def __init__(self):
        # Alternative method to load MNIST, if mldata.org is down
        from scipy.io import loadmat
        mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
        mnist_path = "./mnist-original.mat"
        response = urllib.request.urlopen(mnist_alternative_url)
        with open(mnist_path, "wb") as f:
            content = response.read()
            f.write(content)
        mnist_raw = loadmat(mnist_path)
        self.data = mnist_raw["data"].T
        self.target = mnist_raw["label"][0]

**Task 1a:** Implement the sigmoid function! Make sure it can handle a vector of values.

In [ ]:
import numpy as np

def sigmoid(z):
    # ---------- Add code in between these comments ----------
    # --------------------------------------------------------

**Task 1b:** For later use in the backpropagation algorithm calculate the first derivative of the sigmoid function and implement it.

In [ ]:
def sigmoid_derivative(z):
    # ---------- Add code in between these comments ----------
    # --------------------------------------------------------

**Task 1c:** Plot the implemented sigmoid function and its dericative by executing the next cell to ensure that it works properly!

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

x = np.linspace(-4, 4, num=100)
plt.plot(x, sigmoid(x))
plt.plot(x, sigmoid_derivative(x))
plt.legend(("sigmoid", "derivative"))
plt.show()

The centerpiece is the `Network` class which we use to represent a neural network. The following cell starts to implement the network object by defining the `__init__(self, ...)` method that is the constructor of the class and initializes the member variables. The biases and weights in the Network object are all initialized randomly using a Gaussian distribution with mean 0 and standard deviation 1. This random initialization gives our stochastic gradient descent algorithm a place to start from.

The list `sizes`, which contains the number of neurons in the respective layers of the network, is given as a parameter.  For example, if the list was `[2, 3, 1]`, then it would be a three-layer network with the first layer containing 2 neurons, the second layer 3 neurons, and the third layer 1 neuron. 

After initialization the class instance should contain the following four member variables:

- **`self.num_layers`** is the total number of layers in the network.

- **`self.sizes`** is the same as the parameter `sizes`.

- **`self.biases`** is a list of NumPy vectors. The biases for each hidden and output layer are stored in the respective vector of the list. The biases for the network are initialized randomly, using a Gaussian distribution with mean 0 and variance 1. Note that the first layer is assumed to be an input layer, and by convention we won't set any biases for those neurons, since biases are only ever used in computing the outputs from preceding layers.

- **`self.weights`** is a list of NumPy matrices, each storing the weights of the connection between layers. So for example `self.weights[1]` is a NumPy matrix storing the weights connecting the second and third layers of neurons. (It's not the first and second layers, since Python's list indexing starts at 0.) Since `self.weights[1]` is rather verbose, let's just denote that matrix $w$. It's a matrix such that $w_{jk}$ is the weight for the connection between the $k$-th neuron in the second layer, and the $j$-th neuron in the third layer. 

To generate random values you are supposed to use the NumPy function `np.random.randn()`. Look up the NumPy documentation for further information on the usage.

**Task 2a:** Implement the code initializing the four member variables of the network!


In [ ]:
import jdc # import for jupyter magic command "%%add_to".
import random

class Network(object):

    def __init__(self, sizes):
        """Initialize network with random weights and biases."""
        # ---------- Add code in between these comments ----------
        # --------------------------------------------------------
        
    def __str__(self):
        """String representation of class"""
        return "Number of layers: " + str(self.num_layers) + "\n" + \
               "Sizes of layers: " + str(self.sizes) + "\n" + \
               "Biases:" + "\n" + "\n".join([str(s) for s in self.biases]) + "\n" + \
               "Weights:" + "\n" + "\n".join([str(s) for s in self.weights])

**Task 2b:** In order to verify your implementation, initialize a new `Network` with an input layer of 3 neurons, two hidden layers with 4 neurons each and an output layer of 2 neurons.

In [ ]:
# ---------- Add code in between these comments ----------
# --------------------------------------------------------
print(net)

As mentioned above the weight matrix `self.weights[1]` denoted as $w$ stores the weights such that $w_{jk}$ is the weight for the connection between the $k$-th neuron in the second layer and the $j$-th neuron in the third layer
This ordering of the $j$ and $k$ indices may seem strange - surely it would make more sense to swap the j and k indices around? The big advantage of using this ordering is that it means the vector of activations of the third layer of neurons is:

$$a'=\sigma(wa+b)$$

**Task 3a:** Using the earlier implemented `sigmoid()` function implement the `feed_forward()` method, which applies the equation stated above layer by layer to a given input vector `a`.

Using the built-in `zip()` iterator helps you keep your code cleaner and thereby easier to understand.

In [ ]:
%%add_to Network
def feed_forward(self, a):
    """Return the output of the network if ``a`` is input."""
    # ---------- Add code in between these comments ----------
    # --------------------------------------------------------

**Task 3b:** In order to verify your implementation feed the randomly initialized network from task 2b the input vector `[1.2, 3.4, 5.6]`.

In [ ]:
input = np.array([1.2, 3.4, 5.6]).reshape((-1, 1))
print(net.feed_forward(input))

Now we are having a look at the actual training and validation dataset, which was introduced in the preparation material. 

Using scikit-learn we can easily import the MNIST dataset. After loading, images and labels are accessible in two `numpy.ndarrays`.

- Each of the 70000 rows in `mnist.data` stores one image (28 x 28 pixel) of a handwritten digit as a vector with a length of 784.

- `mnist.target` stores the corresponding labels as an integer number.

**Task 4a:** To familiarize yourself with the data, extract and plot ten different images of each digit (0 - 9).

In [ ]:
mnist = MNIST()

%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 10))

for i in range(0, 10):
    for j in range (0, 10):
        # ---------- Add code in between these comments ----------
        # --------------------------------------------------------
        plt.subplot(10, 10, 10*i+j+1)
        plt.imshow(image, cmap='gist_yarg')
        fig.axes[10*i+j].set_axis_off()
        
plt.show()

Since we want to design our network with 10 output neurons and train it in a way that the output activation for the digit j becomes 1.0 at the j-th neuron, we need to vectorize our label.

**Task 4b:** Implement the function `vectorize()` which accepts an integer $j$ between 0 and 9 and returns a 10-dimensional unit vector with a 1.0 in the $j$-th position and zeroes elsewhere.  This is used to convert a digit (0...9) into a corresponding desired output from the neural network.

In [ ]:
def vectorize(j):
    # ---------- Add code in between these comments ----------
    # --------------------------------------------------------
    
print(vectorize(3))

We need to load and prepare the MNIST dataset to be used in the training of our network. Therefore the following steps need to be taken:

1. Load the data as we did in task 4a and convert images and labels into lists of samples. This is already implemented in the function below.

2. The images in the dataset are integer-valued with a maximum intensity of 255. Before feeding them to the input neuron we have to ensure that their pixel values are between 0.0 and 1.0.

3. The labels in the dataset are integers, so we need to vectorize them using the function implemented above.

4. We have to merge the list of images and the list of labels so that we end up with one list of tuples. Each tuple contains one image and one label.

5. The list of tuples needs to be split into `training_data` (60000 samples) and `validation_data` (10000 samples).

**Task 4c:** Implement steps 2 to 5 in the function below.

In [ ]:

def load_data():
    mnist = MNIST()
    images = [i.reshape((784,1)) for i in mnist.data]
    labels = [int(l) for l in mnist.target]
    
    # ---------- Add code in between these comments ----------
    # --------------------------------------------------------
    
    return training_data, validation_data

training_data, validation_data = load_data()

After each training epoch the performance of the network is evaluated by feeding the validation data into the network and comparing the output to the ground truth label. The neural network's output is assumed to be the index of the neuron with the highest activation in the output layer.

**Task 5a:** Implement the class method `evaluate()`, which feeds the network the validation data sample by sample using the `feed_forward()` method and return the number of correct classifications.

In [ ]:
%%add_to Network
def evaluate(self, validation_data):
    # ---------- Add code in between these comments ----------
    # --------------------------------------------------------

**Task 5b:** Test your implemented code by applying the method to a randomly initialized and untrained network. Since the odds for guessing correctly are 1 to 10 the evaluation output should be roughly 10%. You can execute the code several times to test the assumption.

In [ ]:
training_data, validation_data = load_data()
net = Network([784, 30, 10])
print("Correct classifications of randomly initialized network: {0} of {1}".format(net.evaluate(validation_data), len(validation_data)))

The `train()` method, as its name implies, trains our network. The modification of the the `weights` and `biases` itself is done by the already implemented function `update_mini_batch()`, which again uses the backpropagation algorithm to calculate the gradient.

However, the `train()` method implements the stochastic gradient descent, which means that the gradient is calculated for a subset of training samples, a so called mini batch, rather than for the whole training set. Besides having other advantages this is a lot faster and shortens the time needed for training. After the network has been fed the training set, split into mini batches, and the parameters have been updated accordingly, the process is repeated in the next epoch, thereby iteratively tuning the network parameters to a near-optimal state.

In our implementation of the `train()` method the following steps need to be taken:
- The training and validation data is given formatted as described and impemented in task 4c.
- For every epoch (the number of epochs is given by `num_epochs`):
    - The order of the training data is randomly shuffled.
    - The training dataset is split into mini batches of the size `mini_batch_size`.
    - The `update_mini_batch()` method is applied to each mini batch.
    - The network's perfomance is evaluated and displayed.
    
**Task 6a:** Implement the missing code in the `train()` method as described above.

In [ ]:
%%add_to Network
def train(self, training_data, validation_data, num_epochs, mini_batch_size, learning_rate):
    for j in range(num_epochs):
        # ---------- Add code in between these comments ----------
        # --------------------------------------------------------

def update_mini_batch(self, mini_batch, learning_rate):
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backpropagation(x, y)
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.weights = [w-(learning_rate/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
    self.biases = [b-(learning_rate/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

def backpropagation(self, x, y):
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    # backward pass
    delta = (activations[-1] - y) * sigmoid_derivative(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    for l in range(2, self.num_layers):
        z = zs[-l]
        sp = sigmoid_derivative(z)
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b, nabla_w)

Using all the implemented functionality above you can finally train a network on the MNIST dataset. Therefore:
- Load the training and validation data.
- Instantiate a new network with
    - 784 neurons in the input layer,
    - 30 neurons in the first and only hidden layer and
    - 10 neurons in the output layer.
- Train the network using the training and validation data
    - for 30 epochs,
    - with a mini batch size of 10 and
    - a learning rate of 3.0.
    
**Task 6b:** Implement the training process as stated above.

In [ ]:
# ---------- Add code in between these comments ----------
# --------------------------------------------------------

**Task 7:** Have fun changing the training parameters or the design of the network and observing the resulting classification rate.

In [ ]:
# ---------- Add code in between these comments ----------
# --------------------------------------------------------

### Feedback

Aaaaaand we're done 👏🏼🍻

If you have any suggestions on how we could improve this session, please let us know in the following cell. What did you particularly like or dislike? Did you miss any contents?

### Additional Tasks

**Task 8:** Expand your `train` function by a learning rate decay / scheduler.

In [ ]:
# ---------- Add code in between these comments ----------
# --------------------------------------------------------